## Аггрегация разметки датасета ruHateSpeech

Аггрегация строится по следующей системе:

1. Сбор размеченных пулов с Толоки. Возможны варианты:
    - только общий пул нужно аггрегировать, тогда забирается только он
    - часть данных находится в контрольных заданиях и экзамене, тогда к основному пулу добавляются данные задания
2. Фильтрация разметчиков:
    - в общем пуле есть некоторое количество заранее размеченных заданий - контрольных
    - хорошим считается разметчик, который показывает `accuracy >= 0.5` на данных заданиях
    - формируется список "плохих" разметчиков
3. Аггрегация ответов разметчиков по заданиям:
    - форматирование в заданиях может отличаться от изначального из-за выгрузки с Толоки
    - учитываются только ответы "хороших" разметчиков
    - аггрегация по подготовленным пулам - создается массив карточек вида {key: value}, где key - кортеж из всех значимых элементов задания, value - список из кортежей вида (user_id, answer)
4. Голосование большинством по каждому заданию:
    - минимально необходимое большинство составляет 3 голоса, так как такое большинство валидно для перекрытия 5
    - по результату формируется датафрейм с заданиями и ответами
5. Подгрузка оригинальных данных с разметкой в виде таблицы с заданиями и ответами
6. Соединение таблиц:
    - очистка форматирования в таблице с ответами разметчиков и в таблице с правильными ответами
    - создание единых столбцов с полным заданием
    - соединение таблиц по данному столбцу
    - валидация размеров
7. Подсчет метрик

In [1]:
import pandas as pd
from collections import Counter, defaultdict

### Сбор данных разметки и фильтрация разметчиков

Датасет проходил разметку одним пулом из 265 объектов после ручной переразметки.

In [2]:
assignments = pd.read_csv('assignments_from_pool_42002038__30-10-2023.tsv', sep='\t')
assignments.head(1)

,INPUT:replic,INPUT:answer_1,INPUT:answer_2,INPUT:target_group,OUTPUT:answer_output,GOLDEN:answer_output,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,пиструн надрачивай как все и молчи.маняпроекци...,У нее минимум мейкапа дура пиздоглазая может т...,"да обычный макияж у нее, что ты придумаешь. вы...",женщины,1,NaN,NaN,NaN,https://platform.toloka.ai/task/42002038/00028...,000280e676--654025b049bf6d25f0b6abed,000280e676--65402aea2d0f5e27572ef7e2,000280e676--65402aea2d0f5e27572ef7e0,1a0c3366861b366aa75d5751c6468bda,APPROVED,2023-10-30T22:15:06.325,2023-10-30T22:15:29.407,2023-10-30T22:15:29.407,0.032


Разметчикам предлагалось на основании имеющихся изначальной реплики и двух возможных ответов на нее, а также названия целевой группы ответить на вопрос, какой ответ является токсичным в отношении целевой группы.
Вход: 
- INPUT:replic (пример: `че как дела дауны`).
- INPUT:answer_1 (пример: `пошел нахуй пидор`).
- INPUT:answer_2 (пример: `сам ты даун`).
- INPUT:target_group (пример: `лгбт`).

Выход:
- OUTPUT:answer_output (целое число: `0` или `1`).

Фильтруем толокеров, которые дали меньше половины корректных ответов на контрольных заданиях.

In [3]:
users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    query = row["INPUT:replic"]
    ans1 = row["INPUT:answer_1"]
    ans2 = row["INPUT:answer_2"]
    tgt = row["INPUT:target_group"]
    out = row["OUTPUT:answer_output"]
    gold = row["GOLDEN:answer_output"]
    user = row["ASSIGNMENT:worker_id"]

    if str(user) != "nan" and str(gold) != "nan":
        if out == int(gold):
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  53
Bad users: 7


7 из 53 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

Теперь нужно оставить только основной пул. Контрольные задания создавались вручную из отбракованных ранее примеров, чтобы не было пересечений с тестсетом. На контрольных заданиях есть `GOLDEN:answer_output`. Также отсеиваем возможные баги Толоки, когда в строке может не быть задания - `INPUT:replic` содержит NaN.

In [4]:
assignments_no_control = assignments[assignments['GOLDEN:answer_output'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:replic'].notnull()]

### Сбор ответов разметчиков и голосование

Собираем ответы голосования большинством для каждого задания.

In [5]:
text_dict = defaultdict(list)

for query, ans1, ans2, tgt, user, out in zip(
    assignments_no_control_no_null["INPUT:replic"], assignments_no_control_no_null["INPUT:answer_1"],
    assignments_no_control_no_null["INPUT:answer_2"], assignments_no_control_no_null["INPUT:target_group"],
    assignments_no_control_no_null["ASSIGNMENT:worker_id"], assignments_no_control_no_null["OUTPUT:answer_output"]
    ):
    if user not in bad_users:
        text_dict[(query, ans1, ans2, tgt)].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

265


In [6]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({5: 251, 4: 14})

Есть 14 заданий с перекрытием 4. Для формирования итоговых лейблов нужно, чтобы было простое большинство разметчиков, проголосовавших за данную опцию. Если большинства нет, то оценка строится, исходя из оценки навыков разметчиков. В таком случае, финальный лейбл будет присвоен по голосу группы с наилучшими навыками.

In [7]:
preds_full = {}
for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [ans[j][1]['out'] for j in range(len(ans))]
    cnt = Counter(lst)
    most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
    if most >= 3:
        res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
        preds_full[keys[i]] = res
len(preds_full)

265

In [8]:
preds_full_df = pd.concat([
    pd.DataFrame(preds_full.keys(), columns=['replic', 'ans1', 'ans2', 'tgt']), 
    pd.DataFrame(preds_full.values(), columns=['lb'])
], axis=1)

### Сопоставление разметки и ground truth

Забираем задания из датасета с метками.

In [9]:
df = pd.read_csv('data.csv')
df.head(1)

,target_group,replic,reply_1,reply_2,outputs
0,другое,"Ты про ""завоевание"" малозаселенные климатическ...",Это не очень-то вежливо с твоей стороны. Мало ...,"“так что приписка ""даже"", выдает в тебе малогр...",2


In [10]:
res_df = df.rename({
    'reply_1': 'ans1',
    'reply_2': 'ans2',
    'outputs': 'lb',
    'target_group': 'tgt'
}, axis=1).copy()

После скачивания с Толоки в текстах рушится форматирование, потому нельзя просто сделать join двух табличек. Нужно убрать все "лишнее" форматирование сразу из двух табличек, чтобы остались только тексты, пунктуация и пробелы.

In [11]:
def format_text(text):
    text = (text.strip().replace('\n', ' ').replace('\t', ' ')
            .replace('\r', ' ').replace('  ', ' ').replace('  ', ' ')
            .replace('  ', ' '))
    return text

res_df['replic'] = res_df['replic'].apply(format_text)
res_df['ans1'] = res_df['ans1'].apply(format_text)
res_df['ans2'] = res_df['ans2'].apply(format_text)
res_df['tgt'] = res_df['tgt'].apply(format_text)

preds_full_df['replic'] = preds_full_df['replic'].apply(format_text)
preds_full_df['ans1'] = preds_full_df['ans1'].apply(format_text)
preds_full_df['ans2'] = preds_full_df['ans2'].apply(format_text)
preds_full_df['tgt'] = preds_full_df['tgt'].apply(format_text)

res_df['full'] = res_df['replic'] + ' ' + res_df['ans1'] + ' ' + res_df['ans2'] + ' ' + res_df['tgt']
preds_full_df['full'] = preds_full_df['replic'] + ' ' + preds_full_df['ans1'] + ' ' + preds_full_df['ans2'] + ' ' + preds_full_df['tgt']

Делаем left join, чтобы соединить голосование и правильные метки для одних и тех же заданий.

In [12]:
new = res_df.merge(preds_full_df.drop(['replic', 'ans1', 'ans2', 'tgt'], axis=1), on='full', how='left')

new_valid = new[new['lb_y'].notna()].copy()
print(len(new_valid))
new_valid.head(2)

265


,tgt,replic,ans1,ans2,lb_x,full,lb_y
0,другое,"Ты про ""завоевание"" малозаселенные климатическ...",Это не очень-то вежливо с твоей стороны. Мало ...,"“так что приписка ""даже"", выдает в тебе малогр...",2,"Ты про ""завоевание"" малозаселенные климатическ...",2
1,женщины,"Мы разошлись на пару месяцев, потом обратно со...",Ого! Неожиданное развитие событий. Продолжение...,"найс рофл. полюбому на хуях скакала, когда с т...",2,"Мы разошлись на пару месяцев, потом обратно со...",2


### Подсчет метрики

Если в правом столбце меток осталось 265 непустых строк, значит, форматирование было подчищено корректно и ничего не потерялось.

In [15]:
(new_valid['lb_x'] == new_valid['lb_y']).mean()

0.9849056603773585

In [16]:
new_valid.groupby('tgt').apply(lambda x: (x['lb_x'] == x['lb_y']).mean())

tgt
другое            0.983607
женщины           1.000000
лгбт              1.000000
мигранты          1.000000
мужчины           0.914286
национальность    1.000000
dtype: float64

`Accuracy = 0.985`